In [2]:
from importlib.metadata import metadata

import pandas as pd
import psycopg2
import os
import sys

from panel import Column


In [3]:
# Some neat code I made from our third project, it checks to see if a python script containing variables needed for sql has been made or not.
# If it has not been made, it will create it. You can then proceed to fill out information in the strings.
# This code has had some cross compatibility issues during the project, but it should work.
sqlconfig = os.path.join("ConnectionString","ConnectionString.py")

if not os.path.exists(sqlconfig):
    with open(sqlconfig, 'w') as file:
        file.write(f"connectionString = ''\nconnectionStringLocal = f'postgresql://{{username}}:{{password}}@{{host}}:{{port}}/{{database}}' \nuser = ''\npassword = ''\nhost = ''\nport = ''")
    print(f"Configuration file has been created")
else:
    print(f"Configuration file already exists, importing...")
    
from ConnectionString.ConnectionString import *

Configuration file already exists, importing...


In [5]:
# Creating paths to the CSVs # ignore this was just testing something
departmentsPath = "data/departments.csv"
deptEMPPath = "data/dept_emp.csv"
deptManagerPath = "data/dept_manager.csv"
employeesPath = "data/employees.csv"
salariesPath = "data/salaries.csv"
titlesPath = "data/titles.csv"

departmentsPath = os.path.abspath(departmentsPath)
deptEMPPath = os.path.abspath(deptEMPPath)
deptManagerPath = os.path.abspath(deptManagerPath)
employeesPath = os.path.abspath(employeesPath)
salariesPath = os.path.abspath(salariesPath)
titlesPath = os.path.abspath(titlesPath)

departments = pd.read_csv(departmentsPath)
deptEMP = pd.read_csv(deptEMPPath)
deptManager = pd.read_csv(deptManagerPath)
employees = pd.read_csv(employeesPath)
salaries = pd.read_csv(salariesPath)
titles = pd.read_csv(titlesPath)


In [20]:
# Connection to PostgreSQL server, and to the database I've created
# attaching database name to a string
database = "Module9Challenge"
connection = psycopg2.connect(
    host=host,
    database=database,
    user=username,
    password=password
    )

cur = connection.cursor()


In [21]:
# Creating tables on PostGRESQL server.
cur.execute("""
    CREATE TABLE IF NOT EXISTS titles (
        title_id VARCHAR PRIMARY KEY,
        title VARCHAR
    );
    CREATE TABLE IF NOT EXISTS employees (
        emp_no INT PRIMARY KEY,
        emp_title_id VARCHAR,
        birth_date DATE,
        first_name VARCHAR,
        last_name VARCHAR,
        sex VARCHAR,
        hire_date DATE,
        FOREIGN KEY (emp_title_id) REFERENCES titles (title_id)
    );    
    CREATE TABLE IF NOT EXISTS departments (
        dept_no VARCHAR PRIMARY KEY,
        dept_name VARCHAR
    );
    CREATE TABLE IF NOT EXISTS dept_manager (
        emp_no INT,
        dept_no VARCHAR,
        FOREIGN KEY (emp_no) REFERENCES employees (emp_no),
        FOREIGN KEY (dept_no) REFERENCES departments (dept_no),
        PRIMARY KEY (emp_no, dept_no)
    );
    CREATE TABLE IF NOT EXISTS dept_emp (
        dept_no VARCHAR,
        emp_no INT,
        FOREIGN KEY (dept_no) REFERENCES departments (dept_no),
        FOREIGN KEY (emp_no) REFERENCES employees (emp_no),
        PRIMARY KEY (dept_no, emp_no)
    );
    CREATE TABLE IF NOT EXISTS salaries (
        emp_no INT PRIMARY KEY,
        salary VARCHAR,
        FOREIGN KEY (emp_no) REFERENCES employees (emp_no)
    );
    """)
connection.commit()
connection.close

<function connection.close>